# 1. Import dependencies

In [11]:
#Modules en lien avec Gym
import gymnasium as gym

#Modules utiles
import sys
sys.path.append('/Users/mathiasperez/Documents/GitHub/PSC_Stratolia/Algo Naif')
from parcours import parcours_a_Z, distance_destination, case, ventU_ventV
import Node

import os
import numpy as np
import random
import os
import datetime as dt 
import time
import math

"""
#Modules en lien avec Stable_baseline3
#IMPORT THE TYPE OF AGENT U WANT TO USE
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
"""

'\n#Modules en lien avec Stable_baseline3\n#IMPORT THE TYPE OF AGENT U WANT TO USE\nfrom stable_baselines3.common.vec_env import DummyVecEnv\nfrom stable_baselines3.common.evaluation import evaluate_policy\n'

# 1.bis Import the wind-datas

In [2]:
import pickle
with open("objet_wind_data_2020.pickle", "rb") as f:
    wind_data = pickle.load(f)

"""
Le tableau va de :
datetime.datetime(2020, 1, 1, 0, 0) à datetime.datetime(2021, 1, 1, 0, 0)
"""

'\nLe tableau va de :\ndatetime.datetime(2020, 1, 1, 0, 0) à datetime.datetime(2021, 1, 1, 0, 0)\n'

In [8]:
print((wind_data['metadata']['grid']['time']))

[datetime.datetime(2020, 1, 1, 0, 0), datetime.datetime(2020, 1, 1, 6, 0), datetime.datetime(2020, 1, 1, 12, 0), datetime.datetime(2020, 1, 1, 18, 0), datetime.datetime(2020, 1, 2, 0, 0), datetime.datetime(2020, 1, 2, 6, 0), datetime.datetime(2020, 1, 2, 12, 0), datetime.datetime(2020, 1, 2, 18, 0), datetime.datetime(2020, 1, 3, 0, 0), datetime.datetime(2020, 1, 3, 6, 0), datetime.datetime(2020, 1, 3, 12, 0), datetime.datetime(2020, 1, 3, 18, 0), datetime.datetime(2020, 1, 4, 0, 0), datetime.datetime(2020, 1, 4, 6, 0), datetime.datetime(2020, 1, 4, 12, 0), datetime.datetime(2020, 1, 4, 18, 0), datetime.datetime(2020, 1, 5, 0, 0), datetime.datetime(2020, 1, 5, 6, 0), datetime.datetime(2020, 1, 5, 12, 0), datetime.datetime(2020, 1, 5, 18, 0), datetime.datetime(2020, 1, 6, 0, 0), datetime.datetime(2020, 1, 6, 6, 0), datetime.datetime(2020, 1, 6, 12, 0), datetime.datetime(2020, 1, 6, 18, 0), datetime.datetime(2020, 1, 7, 0, 0), datetime.datetime(2020, 1, 7, 6, 0), datetime.datetime(2020, 1

# 1.bisbis Create needed functions

In [22]:
"""
    INPUT : start_date and end_date -> datetime objects representing the start and end dates
    OUTPUT : A random date bewtween the two inputs and an hour  that is a multiple of 6 (i.e., 0, 6, 12, or 18)
"""
def generate_random_date(start_date, end_date):
    time_between_dates = end_date - start_date
    days_between_dates = time_between_dates.days
    random_number_of_days = random.randrange(days_between_dates)
    random_date = start_date + dt.timedelta(days=random_number_of_days)
    hr  = random.randrange(0, 24, 6)
    random_time = dt.time(hour=hr, minute=0, second=0, microsecond=0, tzinfo=None, fold=0)
    return dt.datetime.combine(random_date, random_time)


"""
INPUT : a set of coordinates, long, lat, time
OUTPUT : a sub-array of wind_data centered around the coordinates we own 
"""
def  get_wind_around_coordinates(longitude, latitude, time, wind_data):
    pass

# 2. Create the environnement

In [23]:
class BalloonEnv(gym.Env):
    """
    ## Description
    This environment is a complete representation of our problem.

    The destination is always at variable coordinates (random opr not).
    So are the starting coordinatesv (probably)


    ## Action Space
    There are three discrete actions available:
    - 0: do nothing et let the wind lead your movement
    - 1: go up in altitude (17 heights allowed)
    - 2: go down in altitude (17 heights allowed)

    ## Observation Space
    The state a tuple consisting of different parts : 
        First you have the time accessible using the datetime module
        Then 2 Boxes (simple array) for the longitude and the latitude
        Finally one discrete part consisting of the 71 different heights

    ## Rewards
    After every step a reward is granted. The total reward of an episode is the
    sum of the rewards for all the steps within that episode.

    For each step, the reward:
    (- is increased/decreased the closer/further the ballon is to the goal point.)
    - is increased/decreased the slower/faster the lander is moving.
    (- is increased/decreased the less/more energy from the battery is used in the round)

    The episode receive an additional reward of +100 or -100 points for finding the goal or not respectively.



    ## Starting State
    The lander starts at a random poinf of the globe (time can be random as well)

    ## Episode Termination
    The episode finishes if:
    1) the balloon gets too far away from the goal;
    2) the balloon goes near enough of the goal

    ## Arguments

     * `goal_long` and 'goal_lat' for the position of our goal destination (can be fixed for specific results)
     * 'initial_time' defines the starting time of our episode (=> consequence on the winds)

    * `precision' defines how close the balloon must be to the target before being considered as having reached it.

    * 'limite_eloignement' defines how far the ballon must be from the target to consider the episode to be a failure.

    * `temps_exploration` defines how long we let the ballon follow the winds when action : 0 is considered

    ## Version History
    - v0: Initial version

    ## Current bugs

    ## Credits
    CMOI
    """
    def __init__(self):
        #super(Balloon, self).__init__()

        #ICI faut mettre None ou deja des random values ?
        self.goal_lat = random.uniform(-90,90)
        self.goal_long = random.uniform(0,360)
        self.initial_time = generate_random_date(dt.datetime(2020, 1, 1, 0, 0), dt.datetime(2021, 1, 1, 0, 0))

        self.limite_eloignement = 10000 #en m
        self.precision = 1000 #en m

        self.winds = self.get_init_winds()
        

        self.temps_exploration = 3600
        
        # Define the action space
        self.action_space = gym.spaces.Discrete(3)  
        """
        Action 0 : You do nothing and let the balloon follow the wind for temps_exploration seconds
        Action 1 : You push the balloon upwards (goes up in altitude) (instanneous action)
        Action 2 : You push the balloon downwards (goes down in altitude) (instanneous action)
        """
        
        # Define the observation space
        self.observation_space = gym.spaces.Tuple(( 
            gym.spaces.Discrete(len(wind_data['metadata']['grid']['time'])*6*3600), # time (in special format)
            gym.spaces.Discrete(18), #Altitude (17 different positions : no need for more precisions)
            gym.spaces.Box(low=0, high=360, shape=(1,), dtype=np.float32),  #Longitude
            gym.spaces.Box(low=-90, high=90, shape=(1,), dtype=np.float32)   #Latitude
            ))
     
    def reset(self):
        # Reset the environment to its initial state
        # Resetting the ballon to a random initial position
        #Ici on pourrait randomiser les goal et les positions de depart +temps
        self.initial_time = generate_random_date(dt.datetime(2020, 1, 1, 0, 0), dt.datetime(2021, 1, 1, 0, 0))
        self.state = ((self.initial_time-dt.datetime(2020, 1, 1, 0, 0)).total_seconds(), 
                      0, 
                      np.array([random.uniform(0,360)], dtype=np.float32),
                      np.array([random.uniform(-90,90)], dtype=np.float32)
                      )
        self.goal_lat = random.uniform(-90,90)
        self.goal_long = random.uniform(0,360)
        while distance_destination((self.goal_long, self.goal_lat), self.state[2], self.state[3]) > 10000000 :
            self.goal_lat = random.uniform(-90,90)
            self.goal_long = random.uniform(0,360)
        self.winds = self.get_init_winds()
        return self.state, {}

    def get_init_winds(self) :
        pass


    
    def get_terminated(self, state):
        long, lat = state[2], state[3]
        if distance_destination((self.goal_long, self.goal_lat), long, lat) < self.limite_eloignement :
            return True
        return False
    
    def get_reward(self, state):
        """
        - fonction qui donne récompense en fonction de la distance  a l'objectif par rapport a la distance initale
        - ajoute un boolean si l'objectif a été atteint ou bien si on en est suffisamment proche
        - d'autres trucs a faire surement
        """
        eloignement = distance_destination((self.goal_long, self.goal_lat), state[2], state[3])
        time  =  state[0].total_seconds() - self.initial_time.total_seconds()
        if (eloignement <= self.limite_eloignement) :
            reward =  1000 + math.log(1/time)
        else :
            reward = math.exp((self.limite_eloignement/eloignement)) +  math.log(1/time)
        return reward

    
    def step(self, action):
        # Take a step in the environment based on the given action
        if action == 0:
            #You decide to follow the wind flow :
            long, lat, pression = self.state[2], self.state[3], self.state[1]
            secondes_depuis_debut = (self.state[0]-self.initial_time).total_seconds()
            temps = secondes_depuis_debut//(6*3600), secondes_depuis_debut%(6*3600)
            n = Node(long, lat,temps, pression, None) #longtiude, latitude, temps, ression, prev
            terminated, n = parcours_a_Z(
                                        (self.goal_long, self.goal_lat),
                                        n,
                                        temps_chgmt_pression = self.temps_exploration,
                                        precision = self.precision, 
                                        tab_vent = wind_data)
            done = terminated
            new_date = n.temps[0]*(6*3600)+n.temps[1]
            self.state = new_date, n.pression, n.long, n.lat

        elif action == 1:
            #You go up 1 case in altitude so u go down in pressure
            self.state[1] -= 1
        else :
            #You go down one case in altitudeso u go up in pressure
            self.state[1] += 1
    

        # Calculate the reward
        reward = get_reward(self.state)
        
        # Check if the episode has ended
        if not done:
            done = get_terminated(self.state)
        
        # Check if the episode has been truncated
        truncated = False
        
        # Return the observation, reward, done flag, truncated flag, and info
        return self.state, reward, done, truncated, {}
    
    def render(self, mode='human'):
        # Render the environment
        pass


    


In [24]:
env = BalloonEnv()

env.observation_space

Tuple(Discrete(31644000), Discrete(18), Box(0.0, 360.0, (1,), float32), Box(-90.0, 90.0, (1,), float32))